In [ ]:
#!/usr/bin/env python3
"""
Basketball Computer Vision Utilities
Integrates YOLO, Roboflow, and OpenCV for basketball analysis
"""
import os
import cv2
import numpy as np
from typing import Dict, List, Tuple, Optional, Any
import supervision as sv
from ultralytics import YOLO
from dataclasses import dataclass
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


@dataclass
class BasketballConfig:
    """Configuration for basketball detection system."""
    # Model paths
    player_model_path: str = "yolov8m.pt"  # For player detection
    ball_model_path: str = "yolov8m.pt"    # Can use custom ball model
    court_model_path: Optional[str] = None  # Roboflow court detection
    
    # Roboflow settings
    roboflow_api_key: Optional[str] = None
    roboflow_workspace: str = "basketball-formations"
    roboflow_project: str = "basketball-court-detection-2-mlopt"
    roboflow_version: int = 1
    
    # Detection thresholds
    player_confidence: float = 0.5
    ball_confidence: float = 0.3
    court_confidence: float = 0.5
    
    # Tracking settings
    track_players: bool = True
    track_ball: bool = True
    max_tracks: int = 20
    
    # Visualization
    show_labels: bool = True
    show_confidence: bool = True
    box_thickness: int = 2
    text_scale: float = 0.5
    
    # Video processing
    target_fps: int = 30
    resize_width: Optional[int] = 1280
    
    def __post_init__(self):
        """Load API key from environment if not provided."""
        if self.roboflow_api_key is None:
            self.roboflow_api_key = os.environ.get('ROBOFLOW_API_KEY')


class BasketballDetector:
    """Main basketball detection and tracking system."""
    
    def __init__(self, config: BasketballConfig = None):
        """Initialize the basketball detector with configuration."""
        self.config = config or BasketballConfig()
        self.models = {}
        self.trackers = {}
        self.court_detector = None
        
        # Initialize models
        self._init_models()
        
        # Initialize trackers
        if self.config.track_players:
            self.trackers['players'] = sv.ByteTrack()
        if self.config.track_ball:
            self.trackers['ball'] = sv.ByteTrack()
        
        # Initialize annotators
        self.box_annotator = sv.BoxAnnotator(
            thickness=self.config.box_thickness,
            text_scale=self.config.text_scale
        )
        self.label_annotator = sv.LabelAnnotator(
            text_scale=self.config.text_scale,
            text_thickness=self.config.box_thickness
        )
    
    def _init_models(self):
        """Initialize YOLO and Roboflow models."""
        try:
            # Load player detection model
            logger.info(f"Loading player model: {self.config.player_model_path}")
            self.models['players'] = YOLO(self.config.player_model_path)
            
            # Load ball detection model (could be same or custom)
            logger.info(f"Loading ball model: {self.config.ball_model_path}")
            self.models['ball'] = YOLO(self.config.ball_model_path)
            
            # Initialize Roboflow court detection if API key available
            if self.config.roboflow_api_key:
                self._init_roboflow()
            else:
                logger.warning("Roboflow API key not set. Court detection disabled.")
                
        except Exception as e:
            logger.error(f"Error initializing models: {e}")
            raise
    
    def _init_roboflow(self):
        """Initialize Roboflow court detection model."""
        try:
            from roboflow import Roboflow
            
            rf = Roboflow(api_key=self.config.roboflow_api_key)
            project = rf.workspace(self.config.roboflow_workspace).project(
                self.config.roboflow_project
            )
            self.court_detector = project.version(self.config.roboflow_version).model
            logger.info("Roboflow court detector initialized")
            
        except Exception as e:
            logger.error(f"Failed to initialize Roboflow: {e}")
            self.court_detector = None
    
    def detect_players(self, frame: np.ndarray) -> sv.Detections:
        """Detect players in the frame."""
        results = self.models['players'](
            frame, 
            conf=self.config.player_confidence,
            classes=[0],  # Person class in COCO
            verbose=False
        )
        
        # Convert to supervision Detections
        detections = sv.Detections.from_ultralytics(results[0])
        
        # Apply tracking if enabled
        if self.config.track_players and 'players' in self.trackers:
            detections = self.trackers['players'].update_with_detections(detections)
        
        return detections
    
    def detect_ball(self, frame: np.ndarray) -> sv.Detections:
        """Detect basketball in the frame."""
        # Using sports ball class (32) from COCO
        # For better results, use a custom basketball model
        results = self.models['ball'](
            frame,
            conf=self.config.ball_confidence,
            classes=[32],  # Sports ball class
            verbose=False
        )
        
        detections = sv.Detections.from_ultralytics(results[0])
        
        # Apply tracking if enabled
        if self.config.track_ball and 'ball' in self.trackers:
            detections = self.trackers['ball'].update_with_detections(detections)
        
        return detections
    
    def detect_court(self, frame: np.ndarray) -> Optional[Dict[str, Any]]:
        """Detect basketball court using Roboflow model."""
        if self.court_detector is None:
            return None
        
        try:
            # Inference on the frame
            result = self.court_detector.predict(
                frame, 
                confidence=self.config.court_confidence
            ).json()
            
            return result
            
        except Exception as e:
            logger.error(f"Court detection failed: {e}")
            return None
    
    def process_frame(self, frame: np.ndarray) -> Tuple[np.ndarray, Dict[str, Any]]:
        """Process a single frame with all detections."""
        # Resize frame if configured
        if self.config.resize_width:
            height, width = frame.shape[:2]
            aspect_ratio = height / width
            new_height = int(self.config.resize_width * aspect_ratio)
            frame = cv2.resize(frame, (self.config.resize_width, new_height))
        
        # Detect objects
        players = self.detect_players(frame)
        ball = self.detect_ball(frame)
        court = self.detect_court(frame) if self.court_detector else None
        
        # Prepare detection results
        detections = {
            'players': players,
            'ball': ball,
            'court': court,
            'frame_shape': frame.shape
        }
        
        # Annotate frame
        annotated_frame = self.annotate_frame(frame, detections)
        
        return annotated_frame, detections
    
    def annotate_frame(self, frame: np.ndarray, detections: Dict[str, Any]) -> np.ndarray:
        """Annotate frame with detection results."""
        annotated = frame.copy()
        
        # Annotate players
        if detections['players'] is not None and len(detections['players']) > 0:
            labels = []
            if self.config.show_labels:
                for i, conf in enumerate(detections['players'].confidence):
                    label = f"Player {i+1}"
                    if self.config.show_confidence:
                        label += f" {conf:.2f}"
                    labels.append(label)
            
            annotated = self.box_annotator.annotate(
                scene=annotated,
                detections=detections['players']
            )
            
            if labels:
                annotated = self.label_annotator.annotate(
                    scene=annotated,
                    detections=detections['players'],
                    labels=labels
                )
        
        # Annotate ball
        if detections['ball'] is not None and len(detections['ball']) > 0:
            ball_labels = []
            if self.config.show_labels:
                for conf in detections['ball'].confidence:
                    label = "Ball"
                    if self.config.show_confidence:
                        label += f" {conf:.2f}"
                    ball_labels.append(label)
            
            # Use different color for ball
            ball_annotator = sv.BoxAnnotator(
                thickness=self.config.box_thickness,
                text_scale=self.config.text_scale,
                color=sv.Color.YELLOW
            )
            
            annotated = ball_annotator.annotate(
                scene=annotated,
                detections=detections['ball']
            )
            
            if ball_labels:
                annotated = self.label_annotator.annotate(
                    scene=annotated,
                    detections=detections['ball'],
                    labels=ball_labels
                )
        
        # Annotate court (if detected)
        if detections['court'] is not None:
            self._annotate_court(annotated, detections['court'])
        
        return annotated
    
    def _annotate_court(self, frame: np.ndarray, court_data: Dict[str, Any]):
        """Annotate court boundaries on the frame."""
        # This depends on the specific format from Roboflow
        # Typically includes keypoints or polygon boundaries
        try:
            if 'predictions' in court_data:
                for pred in court_data['predictions']:
                    if 'points' in pred:
                        # Draw court boundary
                        points = np.array(pred['points'], dtype=np.int32)
                        cv2.polylines(frame, [points], True, (0, 255, 0), 2)
                    elif 'x' in pred and 'y' in pred:
                        # Draw bounding box
                        x, y, w, h = pred['x'], pred['y'], pred['width'], pred['height']
                        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        except Exception as e:
            logger.error(f"Court annotation failed: {e}")
    
    def process_video(self, input_path: str, output_path: str = None,
                     show_preview: bool = False) -> Dict[str, Any]:
        """Process entire video with basketball detection."""
        cap = cv2.VideoCapture(input_path)
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Adjust dimensions if resizing
        if self.config.resize_width:
            aspect_ratio = height / width
            width = self.config.resize_width
            height = int(width * aspect_ratio)
        
        # Setup video writer if output path provided
        writer = None
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        # Process statistics
        stats = {
            'total_frames': total_frames,
            'processed_frames': 0,
            'avg_players_per_frame': [],
            'ball_detected_frames': 0,
            'court_detected_frames': 0
        }
        
        try:
            frame_count = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Process frame
                annotated_frame, detections = self.process_frame(frame)
                
                # Update statistics
                frame_count += 1
                if detections['players'] is not None:
                    stats['avg_players_per_frame'].append(len(detections['players']))
                if detections['ball'] is not None and len(detections['ball']) > 0:
                    stats['ball_detected_frames'] += 1
                if detections['court'] is not None:
                    stats['court_detected_frames'] += 1
                
                # Write frame
                if writer:
                    writer.write(annotated_frame)
                
                # Show preview if requested
                if show_preview:
                    cv2.imshow('Basketball Detection', annotated_frame)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                
                # Progress update
                if frame_count % 30 == 0:
                    logger.info(f"Processed {frame_count}/{total_frames} frames")
            
            stats['processed_frames'] = frame_count
            if stats['avg_players_per_frame']:
                stats['avg_players_per_frame'] = np.mean(stats['avg_players_per_frame'])
            else:
                stats['avg_players_per_frame'] = 0
            
        finally:
            cap.release()
            if writer:
                writer.release()
            if show_preview:
                cv2.destroyAllWindows()
        
        logger.info(f"Video processing complete. Stats: {stats}")
        return stats


# Utility functions for basketball analysis

def calculate_player_speed(tracks: Dict[int, List[Tuple[int, int]]], 
                          fps: int = 30) -> Dict[int, float]:
    """Calculate player speeds from tracking data."""
    speeds = {}
    for track_id, positions in tracks.items():
        if len(positions) > 1:
            distances = []
            for i in range(1, len(positions)):
                dx = positions[i][0] - positions[i-1][0]
                dy = positions[i][1] - positions[i-1][1]
                distances.append(np.sqrt(dx**2 + dy**2))
            
            # Average speed in pixels per second
            avg_distance = np.mean(distances)
            speeds[track_id] = avg_distance * fps
    
    return speeds


def detect_3_second_violation(player_positions: List[Tuple[int, int]], 
                             paint_area: Tuple[int, int, int, int],
                             fps: int = 30) -> bool:
    """Detect if a player has been in the paint for more than 3 seconds."""
    x1, y1, x2, y2 = paint_area
    time_in_paint = 0
    
    for x, y in player_positions:
        if x1 <= x <= x2 and y1 <= y <= y2:
            time_in_paint += 1
        else:
            time_in_paint = 0
        
        if time_in_paint >= 3 * fps:  # 3 seconds at given FPS
            return True
    
    return False


def calibrate_court_homography(court_keypoints: np.ndarray,
                               reference_court: np.ndarray) -> np.ndarray:
    """Calculate homography matrix for court calibration."""
    if len(court_keypoints) < 4 or len(reference_court) < 4:
        raise ValueError("Need at least 4 points for homography")
    
    # Calculate homography matrix
    H, _ = cv2.findHomography(court_keypoints, reference_court, cv2.RANSAC)
    return H


def transform_to_2d_court(positions: np.ndarray, 
                         homography: np.ndarray) -> np.ndarray:
    """Transform player positions to 2D court coordinates."""
    if positions.shape[1] == 2:
        # Add homogeneous coordinate
        ones = np.ones((positions.shape[0], 1))
        positions = np.hstack([positions, ones])
    
    # Apply homography
    transformed = (homography @ positions.T).T
    
    # Normalize by homogeneous coordinate
    transformed = transformed[:, :2] / transformed[:, 2:3]
    
    return transformed


if __name__ == "__main__":
    # Example usage
    config = BasketballConfig(
        player_confidence=0.5,
        ball_confidence=0.3,
        track_players=True,
        track_ball=True,
        show_labels=True,
        show_confidence=True
    )
    
    detector = BasketballDetector(config)
    
    # Test with a sample image
    test_image = np.random.randint(0, 255, (720, 1280, 3), dtype=np.uint8)
    annotated, detections = detector.process_frame(test_image)
    
    print(f"Test completed. Detected {len(detections['players'])} players")
    
    # For video processing:
    # stats = detector.process_video("input_video.mp4", "output_video.mp4")
    # print(f"Video stats: {stats}")

In [ ]:



#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Basketball Computer Vision Demo Notebook
========================================
This notebook demonstrates how to use YOLO, Roboflow, and OpenCV
for basketball game analysis including:
- Court detection
- Player tracking
- Ball detection
- 3-second violation detection
- Speed analysis
"""

# %% [markdown]
# # Basketball Computer Vision Analysis
# 
# This notebook demonstrates the integration of YOLO, Roboflow, and OpenCV
# for comprehensive basketball game analysis.

# %% 
# Import required libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video, Image, display
import supervision as sv
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

# Import our custom basketball utilities
import sys
sys.path.append('/workspace')
try:
    from basketball_utils import BasketballDetector, BasketballConfig
except ImportError:
    print("Note: basketball_utils.py not found. Using inline definitions.")

# %% [markdown]
# ## 1. Environment Setup and Verification

# %%
def check_environment():
    """Check if all required components are installed."""
    import subprocess
    
    checks = {
        "Python": sys.version,
        "OpenCV": cv2.__version__,
        "CUDA Available": False,
        "GPU Device": "None",
        "Roboflow API Key": "Not Set",
        "FFmpeg": "Not Found"
    }
    
    # Check CUDA
    try:
        import torch
        checks["CUDA Available"] = torch.cuda.is_available()
        if checks["CUDA Available"]:
            checks["GPU Device"] = torch.cuda.get_device_name(0)
    except:
        pass
    
    # Check Roboflow API
    if os.environ.get('ROBOFLOW_API_KEY'):
        checks["Roboflow API Key"] = "Set (hidden)"
    
    # Check FFmpeg
    try:
        result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
        if result.returncode == 0:
            checks["FFmpeg"] = "Available"
    except:
        pass
    
    # Display results
    print("=" * 50)
    print("ENVIRONMENT CHECK")
    print("=" * 50)
    for key, value in checks.items():
        print(f"{key:20}: {value}")
    print("=" * 50)
    
    return checks

env_status = check_environment()

# %% [markdown]
# ## 2. Initialize Basketball Detector

# %%
# Configure the basketball detector
config = BasketballConfig(
    # Model settings
    player_model_path="yolov8m.pt",  # Medium model for better accuracy
    ball_model_path="yolov8m.pt",
    
    # Detection thresholds
    player_confidence=0.5,
    ball_confidence=0.3,
    court_confidence=0.5,
    
    # Tracking settings
    track_players=True,
    track_ball=True,
    
    # Visualization
    show_labels=True,
    show_confidence=True,
    box_thickness=2,
    text_scale=0.5,
    
    # Video processing
    resize_width=1280
)

# Initialize detector
print("Initializing Basketball Detector...")
detector = BasketballDetector(config)
print("Detector initialized successfully!")

# %% [markdown]
# ## 3. Download Sample Basketball Video (if needed)

# %%
def download_sample_video():
    """Download a sample basketball video for testing."""
    import yt_dlp
    
    # Sample basketball video URL (replace with actual URL)
    video_url = "https://www.youtube.com/watch?v=SAMPLE_VIDEO_ID"
    
    output_dir = "/workspace/videos/input"
    os.makedirs(output_dir, exist_ok=True)
    
    ydl_opts = {
        'format': 'best[height<=720]',
        'outtmpl': f'{output_dir}/basketball_sample.mp4',
        'quiet': True,
        'no_warnings': True,
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print(f"Downloading sample video from {video_url}...")
            ydl.download([video_url])
            print("Download complete!")
            return f'{output_dir}/basketball_sample.mp4'
    except Exception as e:
        print(f"Download failed: {e}")
        print("Please provide your own basketball video.")
        return None

# Uncomment to download sample
# sample_video = download_sample_video()

# %% [markdown]
# ## 4. Process Single Frame Analysis

# %%
def analyze_frame(image_path=None):
    """Analyze a single frame for basketball elements."""
    
    if image_path and os.path.exists(image_path):
        frame = cv2.imread(image_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    else:
        # Create a sample frame for demonstration
        print("Using synthetic frame for demonstration")
        frame = np.random.randint(0, 255, (720, 1280, 3), dtype=np.uint8)
    
    # Process the frame
    annotated_frame, detections = detector.process_frame(frame)
    
    # Display results
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    
    axes[0].imshow(frame)
    axes[0].set_title("Original Frame")
    axes[0].axis('off')
    
    axes[1].imshow(annotated_frame)
    axes[1].set_title("Detected Elements")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detection statistics
    print("\nDetection Results:")
    print(f"Players detected: {len(detections['players']) if detections['players'] else 0}")
    print(f"Ball detected: {len(detections['ball']) if detections['ball'] else 0}")
    print(f"Court detected: {'Yes' if detections['court'] else 'No'}")
    
    return annotated_frame, detections

# Analyze a frame
annotated, detections = analyze_frame()

# %% [markdown]
# ## 5. Video Processing Pipeline

# %%
def process_basketball_video(input_path, output_path=None, max_frames=100):
    """Process a basketball video with full detection pipeline."""
    
    if not os.path.exists(input_path):
        print(f"Video not found: {input_path}")
        return None
    
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = min(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), max_frames)
    
    print(f"Processing video: {input_path}")
    print(f"FPS: {fps}, Processing {total_frames} frames")
    
    # Storage for tracking data
    player_tracks = {}
    ball_positions = []
    
    # Process frames
    frame_count = 0
    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process frame
        annotated_frame, detections = detector.process_frame(frame)
        
        # Store tracking data
        if detections['players'] is not None and hasattr(detections['players'], 'tracker_id'):
            for i, track_id in enumerate(detections['players'].tracker_id):
                if track_id not in player_tracks:
                    player_tracks[track_id] = []
                bbox = detections['players'].xyxy[i]
                center = ((bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2)
                player_tracks[track_id].append(center)
        
        if detections['ball'] is not None and len(detections['ball']) > 0:
            ball_bbox = detections['ball'].xyxy[0]
            ball_center = ((ball_bbox[0] + ball_bbox[2]) / 2, 
                          (ball_bbox[1] + ball_bbox[3]) / 2)
            ball_positions.append(ball_center)
        
        frame_count += 1
        
        # Progress update
        if frame_count % 30 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")
    
    cap.release()
    
    # Analyze tracking data
    print("\n" + "=" * 50)
    print("TRACKING ANALYSIS")
    print("=" * 50)
    print(f"Unique players tracked: {len(player_tracks)}")
    print(f"Ball detection rate: {len(ball_positions)}/{frame_count} frames")
    
    # Calculate player statistics
    if player_tracks:
        avg_track_length = np.mean([len(track) for track in player_tracks.values()])
        print(f"Average track length: {avg_track_length:.1f} frames")
    
    return player_tracks, ball_positions

# Example usage (requires actual video file)
# tracks, ball_pos = process_basketball_video("/workspace/videos/input/game.mp4")

# %% [markdown]
# ## 6. Advanced Analytics: 3-Second Violation Detection

# %%
def detect_paint_violations(player_tracks, paint_area, fps=30):
    """Detect potential 3-second violations in the paint."""
    
    violations = []
    
    for player_id, positions in player_tracks.items():
        consecutive_frames = 0
        
        for pos in positions:
            x, y = pos
            # Check if player is in paint area
            if (paint_area[0] <= x <= paint_area[2] and 
                paint_area[1] <= y <= paint_area[3]):
                consecutive_frames += 1
                
                # Check for 3-second violation
                if consecutive_frames >= 3 * fps:
                    violations.append({
                        'player_id': player_id,
                        'duration': consecutive_frames / fps,
                        'position': pos
                    })
                    break
            else:
                consecutive_frames = 0
    
    return violations

# Example paint area (would need actual court calibration)
paint_area = (400, 200, 600, 500)  # x1, y1, x2, y2

# Detect violations (requires actual tracking data)
# violations = detect_paint_violations(tracks, paint_area)

# %% [markdown]
# ## 7. Roboflow Court Detection Integration

# %%
def setup_roboflow_court_detection():
    """Setup and test Roboflow court detection model."""
    
    api_key = os.environ.get('ROBOFLOW_API_KEY')
    
    if not api_key:
        print("⚠️ Roboflow API key not set!")
        print("To use court detection:")
        print("1. Get your API key from: https://app.roboflow.com/settings/api")
        print("2. Set it in .env.template file or environment variable")
        return None
    
    try:
        from roboflow import Roboflow
        
        # Initialize Roboflow
        rf = Roboflow(api_key=api_key)
        
        # Access basketball court detection project
        project = rf.workspace("basketball-formations").project(
            "basketball-court-detection-2-mlopt"
        )
        model = project.version(1).model
        
        print("✅ Roboflow court detection model loaded successfully!")
        print("Model: basketball-court-detection-2-mlopt v1")
        
        return model
        
    except Exception as e:
        print(f"❌ Failed to load Roboflow model: {e}")
        return None

court_model = setup_roboflow_court_detection()

# %% [markdown]
# ## 8. Real-time Video Stream Processing (Optional)

# %%
def process_live_stream(camera_index=0, duration=30):
    """Process live video stream from camera."""
    
    cap = cv2.VideoCapture(camera_index)
    
    if not cap.isOpened():
        print("Cannot open camera")
        return
    
    print(f"Starting live stream processing for {duration} seconds...")
    print("Press 'q' to quit early")
    
    import time
    start_time = time.time()
    
    while (time.time() - start_time) < duration:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process frame
        annotated_frame, detections = detector.process_frame(frame)
        
        # Display frame (requires X11 forwarding or display server)
        # cv2.imshow('Basketball Detection', annotated_frame)
        
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Live stream processing complete")

# Uncomment to process live stream (requires camera)
# process_live_stream(camera_index=0, duration=10)

# %% [markdown]
# ## 9. Export and Save Results

# %%
def save_analysis_results(player_tracks, ball_positions, output_dir="/workspace/results"):
    """Save analysis results to files."""
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Save tracking data
    import json
    
    # Convert numpy arrays to lists for JSON serialization
    tracks_data = {
        str(k): [list(pos) for pos in v] 
        for k, v in player_tracks.items()
    }
    
    with open(f"{output_dir}/player_tracks.json", 'w') as f:
        json.dump(tracks_data, f, indent=2)
    
    with open(f"{output_dir}/ball_positions.json", 'w') as f:
        json.dump([list(pos) for pos in ball_positions], f, indent=2)
    
    print(f"Results saved to {output_dir}")
    
    # Generate summary report
    report = f"""
Basketball Analysis Report
========================
Total Players Tracked: {len(player_tracks)}
Ball Detection Rate: {len(ball_positions)} detections
Average Player Track Length: {np.mean([len(t) for t in player_tracks.values()]):.1f} frames

Data saved to: {output_dir}
- player_tracks.json
- ball_positions.json
    """
    
    with open(f"{output_dir}/analysis_report.txt", 'w') as f:
        f.write(report)
    
    print(report)

# Example usage (requires actual analysis data)
# save_analysis_results(tracks, ball_pos)

# %% [markdown]
# ## 10. Visualization of Results

# %%
def visualize_player_heatmap(player_tracks, court_dims=(1280, 720)):
    """Create a heatmap of player positions."""
    
    if not player_tracks:
        print("No tracking data available")
        return
    
    # Create heatmap
    heatmap = np.zeros(court_dims[::-1])
    
    for positions in player_tracks.values():
        for x, y in positions:
            if 0 <= int(x) < court_dims[0] and 0 <= int(y) < court_dims[1]:
                heatmap[int(y), int(x)] += 1
    
    # Apply Gaussian blur for smoother visualization
    heatmap = cv2.GaussianBlur(heatmap, (21, 21), 0)
    
    # Normalize
    if heatmap.max() > 0:
        heatmap = heatmap / heatmap.max()
    
    # Display
    plt.figure(figsize=(12, 7))
    plt.imshow(heatmap, cmap='hot', interpolation='nearest')
    plt.colorbar(label='Player Density')
    plt.title('Player Movement Heatmap')
    plt.xlabel('Court Width')
    plt.ylabel('Court Height')
    plt.show()
    
    return heatmap

# Generate heatmap (requires actual tracking data)
# heatmap = visualize_player_heatmap(tracks)

# %% [markdown]
# ## Summary
# 
# This notebook demonstrated:
# 1. ✅ Environment setup and verification
# 2. ✅ Basketball detector initialization
# 3. ✅ Single frame analysis
# 4. ✅ Video processing pipeline
# 5. ✅ Advanced analytics (3-second violations)
# 6. ✅ Roboflow court detection integration
# 7. ✅ Real-time stream processing
# 8. ✅ Results export and visualization
# 
# ### Next Steps:
# - Add custom basketball model training
# - Implement shot detection and classification
# - Add team identification via jersey colors
# - Integrate play-by-play analysis
# - Add performance metrics dashboard